# CZ Ramsey measurement example

Similar to a $T_2^*$ Ramsey experiment, a CZ Ramsey experiment measures fringes resulting from induced Z rotations, which can result in non-unitary CZs. To rectify this non-unitarity, we determine the correction we need to apply to each qubit in the form of `RZ` rotations. If a CZ is perfectly unitary (or has been compensated for adequately with `RZ` pulses), a CZ Ramsey experiment should return 0 radians for each qubit. If, however, some correction is required, these angles will be non-zero. 

A sample QUIL program at one data point (specified by the equatorial Z rotation which maximizes excited state visibility when equal to the required `RZ` correction) between qubits 0 and 1 would look like 

    DECLARE ro BIT[1]
    DECLARE theta REAL[1]
    RX(pi/2) 0
    CZ 0 1
    RZ(theta) 0
    RX(pi/2) 0
    MEASURE 0 ro[0]
    
Since we can natively parameterize `RZ` rotations continuously on our hardware, we are able to make use of [Quil 2.0's parametric program compilation](https://pyquil.readthedocs.io/en/stable/migration3-declare.html?highlight=parametric%20program%20compilation#Parametric-compilation) using `DECLARE theta REAL[1]` and `RZ(theta) 0`. 

## setup

In [ ]:
from typing import Tuple

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from pyquil.api import get_qc, QuantumComputer
from forest_benchmarking.qubit_spectroscopy import (
    fit_to_sinusoidal_waveform,
    get_peak_from_fit_params,
    run_cz_phase_ramsey,
    sinusoidal_waveform)

## measurement, analysis, and plotting

This method takes in a `pandas.DataFrame` object with all the CZ Ramsey data, including the phases and qubits measured, and plots the CZ Ramsey fringes of each qubit with respect to increasing applied contrast phase. It fits to a sinusoid, evaluates the phases at which excited state visibility is maximized for each qubit, $\theta_0$ and $\theta_1$, and plots a vertical line at these maximum excited state visibility points over the experimental data. 

In [ ]:
def plot_cz_phase_ramsey_fringes(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """
    Plot Ramsey experimental data, the fitted sinusoid, and the maximum of that sinusoid.

    :param df: Experimental results to plot and fit exponential decay curve to.
    :return: None
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    qubits = df['qubit'].unique()
    for idx, qubit in enumerate(qubits):
        qubit_df = df[df['qubit'] == qubit].sort_values('phase')
        phases = qubit_df['phase']
        excited_state_visibilities = qubit_df['avg']

        # plot raw data
        axes[idx].plot(phases, excited_state_visibilities, 'o', label=f"QC{qubit} CZ Ramsey data")

        try:
            # fit to sinusoid
            fit_params, fit_params_errs = fit_to_sinusoidal_waveform(phases,
                                                                     excited_state_visibilities)
        except RuntimeError:
            print(f"Could not fit to experimental data for QC{qubit}")
        else:
            # find max excited state visibility (ESV) and propagate error from fit params
            max_ESV, max_ESV_err = get_peak_from_fit_params(fit_params, fit_params_errs)

            # overlay fitted curve and vertical line at maximum ESV
            axes[idx].plot(phases, sinusoidal_waveform(phases, *fit_params),
                           label=f"QC{qubit} fitted line")
            axes[idx].axvline(max_ESV,
                              label=f"QC{qubit} max ESV={max_ESV:.3f}+/-{max_ESV_err:.3f} rad")

        axes[idx].set_xlabel("Phase on second +X/2 pulse [rad]")
        axes[idx].set_ylabel("Excited State Visibility (ESV)")
        axes[idx].set_title(f"CZ Phase Ramsey fringes on QC{qubit}\n"
                            f"due to CZ_{min(qubits)}_{max(qubits)} application")
        axes[idx].legend(loc='best')

    plt.savefig('ramsey.png')
    plt.show()

    return fit_params, fit_params_errs

In [ ]:
def main(qc: QuantumComputer):
    qubits = qc.qubits()

    start_phase = -1.5 * np.pi
    stop_phase = 1.5 * np.pi
    num_phases = 15
    phases = np.linspace(start_phase, stop_phase, num_phases)

    results = run_cz_phase_ramsey(qc, tuple(qubits[:2]),
                                  start_phase, stop_phase, num_phases,
                                  num_shots=1000, filename='ramsey.json')
    plot_cz_phase_ramsey_fringes(results)

In [ ]:
main(get_qc('9q-square-qvm'))